## Import Python Packages

In [1]:
import pandas as pd
import scanpy as sc
import tarfile

In [2]:
# RPy2 converter from AnnData to SingleCellExperiment and vice versa
# https://github.com/theislab/anndata2ri
import anndata2ri

# activate the conversion before you load the extension:
anndata2ri.activate()

%load_ext rpy2.ipython

## Import R Packages

In [3]:
%%R

suppressPackageStartupMessages(library(data.table))

In [4]:
%%R

sessionInfo()

R version 4.1.0 (2021-05-18)
Platform: x86_64-conda-linux-gnu (64-bit)
Running under: Ubuntu 20.04.2 LTS

Matrix products: default
BLAS/LAPACK: /opt/conda/lib/libopenblasp-r0.3.17.so

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=en_US.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=en_US.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_US.UTF-8 LC_IDENTIFICATION=C       

attached base packages:
[1] tools     stats     graphics  grDevices utils     datasets  methods  
[8] base     

other attached packages:
[1] data.table_1.14.0

loaded via a namespace (and not attached):
[1] compiler_4.1.0


## Parameters

In [5]:
path_h5ad = "/data/processed_data.h5ad"
cluster_obs_name = "Clusters"

## Load h5ad

In [6]:
adata = sc.read_h5ad(path_h5ad)

In [7]:
adata.obs[cluster_obs_name]

120703424257380     9
120703455455644     3
120703455480742    15
120726897191836     0
120726912354163     3
                   ..
241114562058980    10
241114576546654     2
241114576709476     0
241114589615901     1
241114589645541     1
Name: Clusters, Length: 2610, dtype: category
Categories (17, int64): [0, 1, 2, 3, ..., 13, 14, 15, 16]

In [8]:
clusters = adata.obs[cluster_obs_name].unique().sort_values().tolist()
clusters

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]

## Create SingleCellExperiment Object

In [9]:
# since we want to compute differentially expressed genes comparing a cluster to rest of the data,
# we supply the cluster labels as part of sce_v2

# note: Python saves cluster assignments as categorical, but R rejects numeric as categories
# to avoid this, convert the clusters into a list (originally a pandas categories)

sce_v2 = sc.AnnData(
    X = adata.X, 
    obs = pd.DataFrame(
        {'cluster_label': [j for j in adata.obs[cluster_obs_name]]}, 
        index = adata.obs.index
    ),
    var = pd.DataFrame(index = adata.var.index)
)

In [10]:
sce_v2

AnnData object with n_obs × n_vars = 2610 × 15199
    obs: 'cluster_label'

In [11]:
# -i implies we are supplying sce as an input to R
%R -i sce_v2

In [12]:
# set up a single cell experiment object in R, using the raw data stored in 'X' in sce anndata object
%R counts(sce_v2) <- assay(sce_v2, "X"); 
print("Finished Setup")

Finished Setup


In [13]:
%R assay(sce_v2)

array([[2.06040192, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [14]:
%R head(colData(sce_v2))

,cluster_label
120703424257380,9
120703455455644,3
120703455480742,15
120726897191836,0
120726912354163,3
120726943845173,9


In [15]:
%R head(rowData(sce_v2))

""
GNAI3
CDC45
APOH
NARF
CAV2
KLF6


## Prepare Pairwise Data

In [16]:
%%R 

for (j in unique(colData(sce_v2)$cluster_label)) {
    print(paste0('Preparing for cluster ', toString(j)))
    
    # identify cells belonging a specific cluster
    id_cells <- which(colData(sce_v2)$cluster_label == j)
    
    # identify cells belonging to rest of the clusters
    id_cells_rest <- which(colData(sce_v2)$cluster_label != j)
    
    # Create two dataframes: 
    df1 <- t(data.frame(counts(sce_v2)[, id_cells])) # transpose because in sce genes are rows
    df2 <- t(data.frame(counts(sce_v2)[, id_cells_rest])) # transpose because in sce genes are rows
    
    # save as RDS
    saveRDS(df1, file = paste0("df.", toString(j), ".RDS"))
    saveRDS(df2, file = paste0("df.", toString(j), ".rest.RDS"))
}

[1] "Preparing for cluster 9"
[1] "Preparing for cluster 3"
[1] "Preparing for cluster 15"
[1] "Preparing for cluster 0"
[1] "Preparing for cluster 2"
[1] "Preparing for cluster 5"
[1] "Preparing for cluster 7"
[1] "Preparing for cluster 8"
[1] "Preparing for cluster 1"
[1] "Preparing for cluster 6"
[1] "Preparing for cluster 10"
[1] "Preparing for cluster 4"
[1] "Preparing for cluster 11"
[1] "Preparing for cluster 13"
[1] "Preparing for cluster 12"
[1] "Preparing for cluster 14"
[1] "Preparing for cluster 16"


## Tar Gzip

In [17]:
for cluster in clusters:
    print(f"Writing df.{cluster}.tgz")
    with tarfile.open(f"df.{cluster}.tgz", "w:gz") as tar:
        tar.add(f"df.{cluster}.RDS")
        tar.add(f"df.{cluster}.rest.RDS")        

Writing df.0.tgz
Writing df.1.tgz
Writing df.2.tgz
Writing df.3.tgz
Writing df.4.tgz
Writing df.5.tgz
Writing df.6.tgz
Writing df.7.tgz
Writing df.8.tgz
Writing df.9.tgz
Writing df.10.tgz
Writing df.11.tgz
Writing df.12.tgz
Writing df.13.tgz
Writing df.14.tgz
Writing df.15.tgz
Writing df.16.tgz
